In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.preprocessing import StandardScaler
from joblib import dump

# Load and preprocess data

In [ ]:
data=pd.read_csv('../data/raw/student-mat.csv')

# Dropping irrelevant columns and handling categorical variables

In [ ]:
data.drop(columns=['school','sex','address'],inplace=True)
data=pd.get_dummies(data,drop_first=True)

# Splitting into features and target

In [ ]:
X=data.drop(columns=['G3'])
y=data['G3']

# Train-test split

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

# Scaling the data

In [ ]:
scaler=StandardScaler()
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test)

# Multiple models for experimentation

In [ ]:
models={
    "Linear Regression":LinearRegression(),
    "Ridge Regression":Ridge(alpha=1.0),
    "Lasso Regression":Lasso(alpha=0.1),
    "Random Forest":RandomForestRegressor(n_estimators=100,random_state=42)
}

# Training and evaluation

In [ ]:
model_performance={}
for name,model in models.items():
    print(f"\nTraining {name}...")
    scores=cross_val_score(model,X_train_scaled,y_train,cv=5,scoring='neg_mean_squared_error')
    avg_rmse=np.sqrt(-scores.mean())#Cross-validation
    model.fit(X_train_scaled,y_train) #Train model
    y_pred=model.predict(X_test_scaled) #Predictions
    mse=mean_squared_error(y_test,y_pred)# Performance metrics
    rmse=np.sqrt(mse)
    r2=r2_score(y_test,y_pred)
    
    print(f"Model: {name}")
    print(f"Cross-Validated RMSE: {avg_rmse}")
    print(f"Test RMSE: {rmse}")
    print(f"R^2 Score: {r2}")
    
    model_performance[name]={'RMSE':rmse,'R2':r2}

# Comparison of model performances

In [ ]:
performance_df=pd.DataFrame(model_performance).T
print("\nModel Performance Summary:")
print(performance_df)

# Saving the Random Forest model

In [ ]:
best_model=models["Random Forest"]
dump(best_model,'../models/final_model.pkl')
dump(scaler,'../models/scaler.pkl')
print("\nBest model saved to '../models/final_model.pkl'")